In [1]:
import pandas as pd
import numpy as np
from rich.console import Console
from rich.progress import track

# Criar console para visualização aprimorada
console = Console()

# Definir caminhos
BASE_PATH = "C:\\Users\\Haroldo Duraes\\Desktop\\GOvGO\\v0\\#DATA\\PNCP\\"
CAT_PATH = BASE_PATH + "CAT\\"
FILE = CAT_PATH + "Lista_CATSER_Corrigida_Acentuada.xlsx"
ABA = "SERVIÇOS"

# Carregar os dados
console.print("[bold yellow]Carregando dados CATSER...[/bold yellow]")
df = pd.read_excel(FILE, sheet_name=ABA)

console.print(f"[green]DataFrame carregado: {df.shape[0]} linhas, {df.shape[1]} colunas[/green]")
console.print("Primeiras linhas:")
display(df.head())

Carregando dados CATSER...

DataFrame carregado: 2823 linhas, 8 colunas

Primeiras linhas:

,Tipo Material Serviço,CodGrupo,Grupo,CodClasse,Classe,CodigoServico,Serviço,Sit Atual Mat Serv
0,Serviço,'-9,NAO SE APLICA,'-7,INVALIDO,15377,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,Ativo
1,Serviço,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,25852,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,Ativo
2,Serviço,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,25860,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PHP,Ativo
3,Serviço,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,25879,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - FR...,Ativo
4,Serviço,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,25887,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...,Ativo


In [2]:
# Tratar casos de "INVALIDO" na coluna "Classe"
console.print("\n[bold yellow]Tratando casos de INVALIDO...[/bold yellow]")
mask_invalido = df['Classe'] == 'INVALIDO'
num_invalidos = mask_invalido.sum()
console.print(f"Registros com Classe INVALIDO: {num_invalidos}")

# Corrigir valores inválidos
df_corrigido = df.copy()
df_corrigido.loc[mask_invalido, 'Classe'] = df_corrigido.loc[mask_invalido, 'Grupo']
df_corrigido.loc[mask_invalido, 'CodClasse'] = df_corrigido.loc[mask_invalido, 'CodGrupo']

# Criar DataFrame simplificado com apenas as colunas necessárias
console.print("\n[bold yellow]Criando DataFrame simplificado...[/bold yellow]")
catser_df = pd.DataFrame()
catser_df['ITEM'] = df_corrigido['Serviço']
catser_df['GRUPO'] = df_corrigido['CodGrupo'].astype(str) + ' - ' + df_corrigido['Grupo']

console.print("[green]DataFrame simplificado criado[/green]")
console.print("Exemplo de dados:")
display(catser_df.head())


Tratando casos de INVALIDO...

Registros com Classe INVALIDO: 688

Criando DataFrame simplificado...

DataFrame simplificado criado

Exemplo de dados:

,ITEM,GRUPO
0,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,'-9 - NAO SE APLICA
1,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
2,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PHP,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
3,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - FR...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...
4,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...,111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃ...


In [ ]:
# Fazer amostragem dos dados e divisão treino/validação
console.print("\n[bold yellow]Criando amostra representativa e divisão treino/validação...[/bold yellow]")

n = 100  # Máximo de itens por grupo
p = 3    # Proporção para treinamento:validação (4:1)

# Preparar os DataFrames finais
training_df = pd.DataFrame(columns=['ITEM', 'CATEGORIA'])
validation_df = pd.DataFrame(columns=['ITEM', 'CATEGORIA'])

# Função para processar cada grupo
def sample_and_split_group(group_name, group_data):
    global training_df, validation_df
    
    # Se o grupo tem mais itens que n, fazer amostragem
    if len(group_data) > n:
        sampled_group = group_data.sample(n=n, random_state=42)
    else:
        sampled_group = group_data
    
    # Adicionar a categoria formatada
    category = "SERVIÇO; " + group_name
    
    # Caso especial: grupo com apenas 1 item
    if len(sampled_group) == 1:
        # Adicionar o mesmo item em ambos os conjuntos
        single_item = sampled_group['ITEM'].iloc[0]
        
        # Adicionar ao treinamento
        train_data = pd.DataFrame({
            'ITEM': [single_item],
            'CATEGORIA': [category]
        })
        training_df = pd.concat([training_df, train_data], ignore_index=True)
        
        # Adicionar à validação
        val_data = pd.DataFrame({
            'ITEM': [single_item],
            'CATEGORIA': [category]
        })
        validation_df = pd.concat([validation_df, val_data], ignore_index=True)
        
        
    else:
        # Caso normal: dividir na proporção p:1
        total_items = len(sampled_group)
        train_size = max(1, int(total_items * p / (p + 1)))  # Garantir pelo menos 1 item para treinamento
        
        # Embaralhar os dados antes de dividir
        shuffled_group = sampled_group.sample(frac=1, random_state=42)
        
        # Dividir em treinamento e validação
        train_items = shuffled_group.iloc[:train_size]
        val_items = shuffled_group.iloc[train_size:]
        
        # Adicionar ao treinamento
        train_data = pd.DataFrame({
            'ITEM': train_items['ITEM'].tolist(),
            'CATEGORIA': [category] * len(train_items)
        })
        training_df = pd.concat([training_df, train_data], ignore_index=True)
        
        # Adicionar à validação
        val_data = pd.DataFrame({
            'ITEM': val_items['ITEM'].tolist(),
            'CATEGORIA': [category] * len(val_items)
        })
        validation_df = pd.concat([validation_df, val_data], ignore_index=True)
    
    return sampled_group  # Retorna o grupo amostrado para o sample_df


# Criar sample_df para armazenar todos os itens amostrados
sample_df = pd.DataFrame(columns=catser_df.columns)

# Processar cada grupo
for group_name, group_data in track(catser_df.groupby('GRUPO'), description="Processando grupos"):
    sampled_group = sample_and_split_group(group_name, group_data)
    sample_df = pd.concat([sample_df, sampled_group], ignore_index=True)

# Estatísticas sobre os conjuntos resultantes
console.print(f"\n[green]DataFrame original: {catser_df.shape[0]} itens[/green]")
console.print(f"[green]Amostra total: {sample_df.shape[0]} itens ({sample_df.shape[0]/catser_df.shape[0]*100:.2f}% do original)[/green]")
console.print(f"[green]Conjunto de treinamento: {training_df.shape[0]} itens ({training_df.shape[0]/sample_df.shape[0]*100:.2f}% da amostra)[/green]")
console.print(f"[green]Conjunto de validação: {validation_df.shape[0]} itens ({validation_df.shape[0]/sample_df.shape[0]*100:.2f}% da amostra)[/green]")
console.print(f"[green]Proporção treinamento:validação = {training_df.shape[0]/validation_df.shape[0]:.2f}:1[/green]")

# Número de grupos
num_grupos = training_df['CATEGORIA'].nunique()
console.print(f"[green]Número total de grupos: {num_grupos}[/green]")

# Exemplos
console.print("\nExemplos do conjunto de treinamento:")
display(training_df.head())

console.print("\nExemplos do conjunto de validação:")
display(validation_df.head())

# Distribuição de grupos
console.print("\nDistribuição de itens por grupo (top 10):")
grupo_counts = pd.DataFrame({
    'Treinamento': training_df['CATEGORIA'].value_counts(),
    'Validação': validation_df['CATEGORIA'].value_counts()
}).fillna(0).astype(int)

display(grupo_counts.head(10))

# Salvar resultados
console.print("\n[bold yellow]Salvando resultados...[/bold yellow]")
output_path = f"{CAT_PATH}catser_datasets_nv1.xlsx"

with pd.ExcelWriter(output_path) as writer:
    training_df.to_excel(writer, sheet_name="training", index=False)
    validation_df.to_excel(writer, sheet_name="validation", index=False)
    sample_df.to_excel(writer, sheet_name="sample", index=False)

console.print(f"[bold green]Dados salvos em: {output_path}[/bold green]")

Criando amostra representativa e divisão treino/validação...

Output()

DataFrame original: 2823 itens

Amostra total: 1844 itens (65.32% do original)

Conjunto de treinamento: 1352 itens (73.32% da amostra)

...

,ITEM,CATEGORIA
0,INFRA-ESTRUTURA AEROPORTUARIA - OPERACAO INDUS...,SERVIÇO; '-9 - NAO SE APLICA
1,MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO D...,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...
2,MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO D...,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...
3,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...
4,DESENVOLVIMENTO E/OU EVOLUCAO DE SOFTWATE - OU...,SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E ...


Distribuição de itens por grupo (top 10):

,Treinamento,Validação
CATEGORIA,,
SERVIÇO; '-9 - NAO SE APLICA,1,1
SERVIÇO; 111 - SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE SOFTWARE,10,4
SERVIÇO; 112 - SERVIÇOS DE MANUTENÇÃO E SUSTENTAÇÃO DE SOFTWARE,1,1
SERVIÇO; 113 - SERVIÇOS DE DOCUMENTAÇÃO DE SOFTWARE,1,1
SERVIÇO; 114 - SERVIÇOS SE ENGENHARIA DE REQUISITOS DE SOFTWARE,1,1
SERVIÇO; 115 - SERVIÇOS DE MENSURAÇÃO DE SOFTWARE,1,1
SERVIÇO; 116 - SERVIÇOS DE QUALIDADE DE SOFTWARE,1,1
SERVIÇO; 117 - SERVIÇOS DE IMPLEMENTAÇÃO ÁGIL DE SOFTWARE,1,1
SERVIÇO; 131 - SERVIÇOS DE COMPUTAÇÃO EM NUVEM,2,1


Salvando resultados...

Dados salvos em: C:\Users\Haroldo Duraes\Desktop\GOvGO\v0\#DATA\PNCP\CAT\catser_grupo_datasets.xlsx